In [2]:
import pandas as pd
import datetime as dt
from datetime import date,time,datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
lbl = LabelEncoder()
mlb = MultiLabelBinarizer()
import numpy as np
%matplotlib inline
from sklearn.externals import joblib

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
df = pd.read_pickle("../dataset.pkl")

In [4]:
df

,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,50806,50808,...,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
0,2,0,65,462.0,0.0,24.0,22.0,1.0,104.0,1.12,...,0,0,0,0,0,0,0,1,1,0
1,6,1,71,462.0,0.0,24.0,26.0,1.0,104.0,1.20,...,0,0,0,0,0,0,0,0,0,1
2,7,1,75,462.0,1.0,24.0,27.0,1.0,98.0,1.07,...,0,0,0,0,0,1,0,0,0,1
3,3,1,39,462.0,0.0,24.0,27.0,1.0,104.0,1.12,...,0,0,0,0,0,0,0,0,0,0
4,4,1,59,462.0,0.0,24.0,26.0,1.0,104.0,1.11,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,3,0,20,230.0,-2.0,26.0,25.0,1.0,101.0,1.12,...,0,0,0,0,0,0,0,1,1,0
58972,1,0,84,485.0,-3.0,18.0,18.0,1.0,104.0,1.12,...,0,0,0,0,0,0,0,0,0,1
58973,9,1,42,422.0,-1.0,24.0,27.0,1.0,103.0,1.08,...,0,1,0,1,0,0,0,1,1,1
58974,11,0,60,462.0,-3.0,22.0,23.0,2.0,111.0,1.00,...,0,0,0,0,0,0,0,1,1,0


In [8]:
x = df.iloc[:,:483]
x.head
#483 = 3(adm_days,gender,age)+480(lab values)

<bound method NDFrame.head of        ADMISSION_DAYS  GENDER  AGE  50801  50802  50803  50804  50805  50806  \
0                   2       0   65  462.0    0.0   24.0   22.0    1.0  104.0   
1                   6       1   71  462.0    0.0   24.0   26.0    1.0  104.0   
2                   7       1   75  462.0    1.0   24.0   27.0    1.0   98.0   
3                   3       1   39  462.0    0.0   24.0   27.0    1.0  104.0   
4                   4       1   59  462.0    0.0   24.0   26.0    1.0  104.0   
...               ...     ...  ...    ...    ...    ...    ...    ...    ...   
58971               3       0   20  230.0   -2.0   26.0   25.0    1.0  101.0   
58972               1       0   84  485.0   -3.0   18.0   18.0    1.0  104.0   
58973               9       1   42  422.0   -1.0   24.0   27.0    1.0  103.0   
58974              11       0   60  462.0   -3.0   22.0   23.0    2.0  111.0   
58975               2       0   63  462.0    0.0   24.0   26.0    1.0  104.0   

       50

In [6]:
y = df.iloc[:,483:]


In [7]:
x.columns

Index(['ADMISSION_DAYS',         'GENDER',            'AGE',            50801,
                  50802,            50803,            50804,            50805,
                  50806,            50808,
       ...
                  51512,            51513,            51514,            51515,
                  51516,            51517,            51518,            51519,
                  51529,            51532],
      dtype='object', length=483)

In [7]:
from sklearn.model_selection import train_test_split # seed?
seed=243 
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.15, random_state=seed)

In [8]:
y.shape

(58976, 20)

In [9]:
x_train.shape

(50129, 483)

In [10]:
y_train.shape

(50129, 20)

In [11]:
x_test.shape

(8847, 483)

In [12]:
y_test.shape

(8847, 20)

In [13]:
np.unique(y_train)

array([0, 1])

In [20]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss
sample_weights = [1-3032/58976,1-8924/58976,1-35608/58976,1-19366/58976,
                 1-16271/58976,1-20008/58976,1-43243/58976,
                 1-25281/58976,1-21200/58976,1-21782/58976,1-166/58976,          
                 1-8316/58976,1-10053/58976,1-3205/58976,
                 1-5492/58976,1-19454/58976,1-1757/58976,1-23352/58976,1-16621/58976,1-34350/58976]
weight_rf = [{0:3032/58976,1:1-3032/58976},{0:8924/58976,1:1-8924/58976},{0:1-35608/58976,1:35608/58976},{0:19366/58976,1:1-19366/58976},
            {0:16271/58976,1:1-16271/58976},{0:20008/58976,1:1-20008/58976},{0:1-43243/58976,1:43243/58976},
            {0:25281/58976,1:1-25281/58976},{0:21200/58976,1:1-21200/58976},{0:21782/58976,1:1-21782/58976},{0:166/58976,1:1-166/58976},
            {0:8316/58976,1:1-8316/58976},{0:10053/58976,1:1-10053/58976},{0:3205/58976,1:1-3205/58976},
            {0:5492/58976,1:1-5492/58976},{0:19454/58976,1:1-19454/58976},{0:1757/58976,1:1-1757/58976},
            {0:23352/58976,1:1-23352/58976},{0:16621/58976,1:1-16621/58976},{0:1-34350/58976,1:34350/58976}]*58976
weights = [0.286,0.714]#this for catboost ok?
#[0,1] classes 0.286 is probabbility of 1 in dataset(icd) so i flipped it over as to give more weight to 1

#model1 = CatBoostClassifier(iterations=1000, task_type="GPU",devices='0:1',learning_rate=0.1)
#d_train = lgb.Dataset(x_train, label=y_train)
#clf_multilabel = OneVsRestClassifier(LGBMClassifier(iterations=100))
clf_multilabel = OneVsRestClassifier(CatBoostClassifier(1500,class_weights=weights,task_type="GPU",
                           devices='0:1'))

In [21]:
'''parameters = {
    
    
    "learning_rate":[0.1, 0.2, 0.3, 0.4]
}

model_tunning = GridSearchCV(clf_multilabel, param_grid=parameters,n_jobs=-1,cv=10,
                             scoring = 'roc_auc')

model_tunning.fit(x_train, y_train)

print(model_tunning.best_score_)
print(model_tunning.best_params_)'''

'parameters = {\n    \n    \n    "learning_rate":[0.1, 0.2, 0.3, 0.4]\n}\n\nmodel_tunning = GridSearchCV(clf_multilabel, param_grid=parameters,n_jobs=-1,cv=10,\n                             scoring = \'roc_auc\')\n\nmodel_tunning.fit(x_train, y_train)\n\nprint(model_tunning.best_score_)\nprint(model_tunning.best_params_)'

In [22]:
#estimator.get_params().keys()

In [23]:
cbc = clf_multilabel.fit(x_train,y_train)
joblib.dump(cbc, "cat1-C.pkl")

Learning rate set to 0.035347


0:	learn: 0.6623759	total: 164ms	remaining: 4m 5s
1:	learn: 0.6341898	total: 558ms	remaining: 6m 57s
2:	learn: 0.6090686	total: 770ms	remaining: 6m 24s
3:	learn: 0.5862991	total: 890ms	remaining: 5m 32s
4:	learn: 0.5658497	total: 916ms	remaining: 4m 33s
5:	learn: 0.5445775	total: 952ms	remaining: 3m 57s
6:	learn: 0.5248831	total: 985ms	remaining: 3m 30s
7:	learn: 0.5086486	total: 1.25s	remaining: 3m 54s
8:	learn: 0.4945988	total: 1.59s	remaining: 4m 23s
9:	learn: 0.4815317	total: 1.81s	remaining: 4m 30s
10:	learn: 0.4685941	total: 2.08s	remaining: 4m 41s
11:	learn: 0.4576486	total: 2.19s	remaining: 4m 32s
12:	learn: 0.4466809	total: 2.52s	remaining: 4m 48s
13:	learn: 0.4374155	total: 3.04s	remaining: 5m 23s
14:	learn: 0.4281234	total: 3.17s	remaining: 5m 13s
15:	learn: 0.4197792	total: 3.38s	remaining: 5m 13s
16:	learn: 0.4120451	total: 3.44s	remaining: 5m
17:	learn: 0.4053152	total: 3.5s	remaining: 4m 47s
18:	learn: 0.3984435	total: 3.55s	remaining: 4m 36s
19:	learn: 0.3925584	total: 

158:	learn: 0.2906879	total: 22.9s	remaining: 3m 13s
159:	learn: 0.2905950	total: 23.2s	remaining: 3m 14s
160:	learn: 0.2904750	total: 23.2s	remaining: 3m 12s
161:	learn: 0.2903996	total: 23.6s	remaining: 3m 14s
162:	learn: 0.2903345	total: 23.8s	remaining: 3m 14s
163:	learn: 0.2901866	total: 24s	remaining: 3m 15s
164:	learn: 0.2900958	total: 24.2s	remaining: 3m 15s
165:	learn: 0.2899812	total: 24.3s	remaining: 3m 15s
166:	learn: 0.2898963	total: 24.6s	remaining: 3m 16s
167:	learn: 0.2898194	total: 24.7s	remaining: 3m 15s
168:	learn: 0.2897237	total: 25.1s	remaining: 3m 17s
169:	learn: 0.2896822	total: 25.3s	remaining: 3m 18s
170:	learn: 0.2896118	total: 25.5s	remaining: 3m 18s
171:	learn: 0.2895246	total: 25.8s	remaining: 3m 19s
172:	learn: 0.2893547	total: 26s	remaining: 3m 19s
173:	learn: 0.2892403	total: 26.1s	remaining: 3m 18s
174:	learn: 0.2891726	total: 26.1s	remaining: 3m 17s
175:	learn: 0.2891155	total: 26.5s	remaining: 3m 19s
176:	learn: 0.2889194	total: 26.7s	remaining: 3m 1

315:	learn: 0.2782361	total: 44.8s	remaining: 2m 47s
316:	learn: 0.2781827	total: 45.4s	remaining: 2m 49s
317:	learn: 0.2779833	total: 45.5s	remaining: 2m 49s
318:	learn: 0.2778708	total: 45.6s	remaining: 2m 48s
319:	learn: 0.2777998	total: 45.6s	remaining: 2m 48s
320:	learn: 0.2776870	total: 45.9s	remaining: 2m 48s
321:	learn: 0.2775972	total: 46.2s	remaining: 2m 49s
322:	learn: 0.2775471	total: 46.5s	remaining: 2m 49s
323:	learn: 0.2774986	total: 46.7s	remaining: 2m 49s
324:	learn: 0.2773797	total: 46.8s	remaining: 2m 49s
325:	learn: 0.2772915	total: 47.1s	remaining: 2m 49s
326:	learn: 0.2771785	total: 47.5s	remaining: 2m 50s
327:	learn: 0.2771118	total: 47.8s	remaining: 2m 50s
328:	learn: 0.2769746	total: 48.1s	remaining: 2m 51s
329:	learn: 0.2769343	total: 48.4s	remaining: 2m 51s
330:	learn: 0.2768616	total: 48.6s	remaining: 2m 51s
331:	learn: 0.2767284	total: 49s	remaining: 2m 52s
332:	learn: 0.2766624	total: 49.1s	remaining: 2m 51s
333:	learn: 0.2765817	total: 49.1s	remaining: 2m

473:	learn: 0.2680233	total: 1m 11s	remaining: 2m 33s
474:	learn: 0.2679987	total: 1m 11s	remaining: 2m 33s
475:	learn: 0.2679500	total: 1m 11s	remaining: 2m 33s
476:	learn: 0.2678911	total: 1m 11s	remaining: 2m 33s
477:	learn: 0.2678051	total: 1m 11s	remaining: 2m 33s
478:	learn: 0.2677289	total: 1m 12s	remaining: 2m 34s
479:	learn: 0.2676864	total: 1m 12s	remaining: 2m 34s
480:	learn: 0.2676482	total: 1m 12s	remaining: 2m 34s
481:	learn: 0.2676101	total: 1m 13s	remaining: 2m 34s
482:	learn: 0.2675668	total: 1m 13s	remaining: 2m 34s
483:	learn: 0.2674901	total: 1m 13s	remaining: 2m 33s
484:	learn: 0.2674195	total: 1m 13s	remaining: 2m 34s
485:	learn: 0.2673679	total: 1m 13s	remaining: 2m 33s
486:	learn: 0.2673400	total: 1m 13s	remaining: 2m 33s
487:	learn: 0.2672966	total: 1m 13s	remaining: 2m 32s
488:	learn: 0.2672132	total: 1m 13s	remaining: 2m 32s
489:	learn: 0.2671641	total: 1m 14s	remaining: 2m 32s
490:	learn: 0.2670953	total: 1m 14s	remaining: 2m 32s
491:	learn: 0.2670695	total:

...

990:	learn: 0.3915612	total: 14s	remaining: 7.17s
991:	learn: 0.3914963	total: 14s	remaining: 7.16s
992:	learn: 0.3914536	total: 14s	remaining: 7.14s
993:	learn: 0.3914213	total: 14s	remaining: 7.13s
994:	learn: 0.3914082	total: 14s	remaining: 7.11s
995:	learn: 0.3913764	total: 14s	remaining: 7.1s
996:	learn: 0.3913388	total: 14s	remaining: 7.08s
997:	learn: 0.3912817	total: 14s	remaining: 7.07s
998:	learn: 0.3912379	total: 14.1s	remaining: 7.05s
999:	learn: 0.3912157	total: 14.1s	remaining: 7.04s
1000:	learn: 0.3911778	total: 14.2s	remaining: 7.05s
1001:	learn: 0.3911563	total: 14.2s	remaining: 7.04s
1002:	learn: 0.3911315	total: 14.2s	remaining: 7.02s
1003:	learn: 0.3911110	total: 14.2s	remaining: 7.01s
1004:	learn: 0.3910826	total: 14.2s	remaining: 7s
1005:	learn: 0.3910540	total: 14.2s	remaining: 6.98s
1006:	learn: 0.3910263	total: 14.2s	remaining: 6.97s
1007:	learn: 0.3910023	total: 14.2s	remaining: 6.95s
1008:	learn: 0.3909768	total: 14.3s	remaining: 6.94s
1009:	learn: 0.3909658	

1156:	learn: 0.3866248	total: 16.6s	remaining: 4.93s
1157:	learn: 0.3866180	total: 16.6s	remaining: 4.91s
1158:	learn: 0.3865950	total: 16.6s	remaining: 4.9s
1159:	learn: 0.3865455	total: 16.6s	remaining: 4.88s
1160:	learn: 0.3865255	total: 16.7s	remaining: 4.86s
1161:	learn: 0.3865018	total: 16.7s	remaining: 4.85s
1162:	learn: 0.3864794	total: 16.7s	remaining: 4.83s
1163:	learn: 0.3864309	total: 16.7s	remaining: 4.82s
1164:	learn: 0.3864087	total: 16.7s	remaining: 4.8s
1165:	learn: 0.3863746	total: 16.7s	remaining: 4.79s
1166:	learn: 0.3863225	total: 16.7s	remaining: 4.77s
1167:	learn: 0.3862861	total: 16.7s	remaining: 4.75s
1168:	learn: 0.3862419	total: 16.7s	remaining: 4.74s
1169:	learn: 0.3862211	total: 16.7s	remaining: 4.72s
1170:	learn: 0.3862097	total: 16.8s	remaining: 4.71s
1171:	learn: 0.3861921	total: 16.8s	remaining: 4.69s
1172:	learn: 0.3861523	total: 16.8s	remaining: 4.67s
1173:	learn: 0.3860970	total: 16.8s	remaining: 4.66s
1174:	learn: 0.3860560	total: 16.8s	remaining: 4

1325:	learn: 0.3819167	total: 18.2s	remaining: 2.39s
1326:	learn: 0.3818843	total: 18.2s	remaining: 2.38s
1327:	learn: 0.3818484	total: 18.3s	remaining: 2.36s
1328:	learn: 0.3818246	total: 18.3s	remaining: 2.35s
1329:	learn: 0.3817959	total: 18.3s	remaining: 2.33s
1330:	learn: 0.3817633	total: 18.3s	remaining: 2.32s
1331:	learn: 0.3817339	total: 18.3s	remaining: 2.31s
1332:	learn: 0.3816898	total: 18.3s	remaining: 2.29s
1333:	learn: 0.3816516	total: 18.3s	remaining: 2.28s
1334:	learn: 0.3816174	total: 18.3s	remaining: 2.26s
1335:	learn: 0.3815770	total: 18.3s	remaining: 2.25s
1336:	learn: 0.3815445	total: 18.3s	remaining: 2.23s
1337:	learn: 0.3815102	total: 18.3s	remaining: 2.22s
1338:	learn: 0.3814952	total: 18.4s	remaining: 2.21s
1339:	learn: 0.3814635	total: 18.4s	remaining: 2.19s
1340:	learn: 0.3814373	total: 18.4s	remaining: 2.18s
1341:	learn: 0.3814025	total: 18.4s	remaining: 2.17s
1342:	learn: 0.3813703	total: 18.4s	remaining: 2.15s
1343:	learn: 0.3813336	total: 18.4s	remaining:

1496:	learn: 0.3772496	total: 20.3s	remaining: 40.7ms
1497:	learn: 0.3772251	total: 20.3s	remaining: 27.1ms
1498:	learn: 0.3771988	total: 20.3s	remaining: 13.5ms
1499:	learn: 0.3771624	total: 20.3s	remaining: 0us


['cat1-C.pkl']

In [24]:
#print(clf_multilabel.best_score_)
mod = joblib.load("cat1-C.pkl")

In [25]:
y_pred = mod.predict(x_test)

In [26]:
y_pred

array([[0, 0, 1, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 1, 0, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [0, 0, 1, ..., 1, 1, 1],
       [0, 0, 1, ..., 1, 1, 1]])

In [27]:
#cm = confusion_matrix(y_test,y_pred) 
#Classification metrics can't handle a mix of multilabel-indicator and unknown targets
print(jaccard_score(y_test,y_pred,average='micro'))
print(jaccard_score(y_test,y_pred,average='macro'))
print(roc_auc_score(y_test,y_pred))
print(hamming_loss(y_test,y_pred))


0.5394577252631167
0.43430201852516453
0.691337534365617
0.19957612750084774


In [ ]:
#y_predtest_prob.to_pickle("./y_predtest_probcat.pkl")

In [ ]:
#y_predtrain_prob.to_pickle("./y_predtrain_probcat.pkl")